# Reward luxembourgish instruction tuning set

In this notebook, we reward a synthetically created luxembourgish instruction tuning set, which was created with Llama-3.1-405b-instruct model from luxembourgish Wikipedia and RTL news data. The original dataset contains 2420 rows, 1930 of which were generated from RTL data and 490 from Wikipedia data. We decided however to only evaluate a subset of 200 randomly sampled entries for helpfulness, correctnes and coherence, using Nvidia-Nemotron-340b-reward. We will sample 80% from the RTL data and 20% from the wiki data, keeping approximately the distribution in the original dataset. Since the model only learned to evaluate English language data, we translated the dataset from Luxembourgish into English using Google translate.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# read in the english csv file
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Master Thesis/instruction_tuning_set_en.csv')
df

,instruction,response,text
0,Give the most important information about Debo...,Deborah De Robertis is a Luxembourgish-French ...,The following provides an instruction in the f...
1,What is the most famous performance of Deborah...,The most famous performance of Deborah De Robe...,The following provides an instruction in the f...
2,What is the background of the performance of D...,"On August 31, 2018, Deborah De Robertis posed ...",The following provides an instruction in the f...
3,What was Deborah De Robertis doing at the Cent...,"On May 6, 2024, Deborah De Robertis hung up an...",The following provides an instruction in the f...
4,How was Deborah De Robertis charged in court o...,"On 29 May 2024, Deborah De Robertis was charge...",The following provides an instruction in the f...
...,...,...,...
2415,What was the context of the national debt in t...,"On August 1, 2011, President Barack Obama anno...",The following provides an instruction in the f...
2416,How much should the US debt limit be raised?,The debt limit should be increased in two stag...,The following provides an instruction in the f...
2417,What are the reactions to the agreement?,Not everyone agrees with this compromise. Resi...,The following provides an instruction in the f...
2418,What are the consequences if the agreement is ...,"If the agreement is not accepted, the United S...",The following provides an instruction in the f...


# Sample Data

We split the dataset into two parts. The first part contains all rows that were generated from the wikipedia data, rows 0 to 489 and the second part, contaning all rows that were generated from the RTL data, rows 490 to 2419. We then randomly sample 40 rows from the wiki data and 160 rows from the RTL data, maintaining the 0.2/0.8 split from the original dataset.

In [ ]:
wiki_df = df.iloc[:490]
rtl_df = df.iloc[490:]
print(len(wiki_df))
print(len(rtl_df))

490
1930


In [ ]:
# random sample 40 rows from wiki data
sampled_wiki_df = wiki_df.sample(n=40, random_state=42)
sampled_wiki_df = sampled_wiki_df.reset_index(drop=True) # reset index and drop orignal index
sampled_wiki_df

,instruction,response,text
0,What is the brightest star in Dorado?,"α Doradus, the brightest star in Dorado, is 20...",The following provides an instruction in the f...
1,"What happened to the Edict of Nantes, after He...","The Edict of Nantes was unfortunately not ""per...",The following provides an instruction in the f...
2,When did the devolution war start and how did ...,The war of devolution began in the spring of 1...,The following provides an instruction in the f...
3,When and where did Max Ophüls die?,"Max Ophüls died on March 26, 1957 in Hamburg.",The following provides an instruction in the f...
4,What happened to the Schumacher building after...,The police museum of the non-profit organizati...,The following provides an instruction in the f...
5,What are the essential amino acids? List the 9...,"The essential amino acids are: histidine, isol...",The following provides an instruction in the f...
6,Give me the names of the former municipalities...,"The former municipalities of Beho, Bovigny, Ch...",The following provides an instruction in the f...
7,Give me the area of the Polvermillen and what ...,"The Polvermillen has an area of 24.82 ha, whic...",The following provides an instruction in the f...
8,When did Victor Tedesco start his prison sente...,After the failed expedition of Exilbelsch in t...,The following provides an instruction in the f...
9,How many films does the Cinémathèque de la Vil...,The Cinémathèque de la Ville de Luxembourg pro...,The following provides an instruction in the f...


In [ ]:
# now same for RTL data, but with 160 rows
sampled_rtl_df = rtl_df.sample(n=160, random_state=42)
sampled_rtl_df = sampled_rtl_df.reset_index(drop=True)
sampled_rtl_df

,instruction,response,text
0,When did Marco Mille work at Siemens?,Marco Mille only joined Siemens in March 2010.,The following provides an instruction in the f...
1,What is the result of the large-scale police c...,The police issued 20 warning charges and one p...,The following provides an instruction in the f...
2,Give me the number of young people in Luxembou...,"In 2019, 9.75 percent of children were fundame...",The following provides an instruction in the f...
3,What did the owner of an animal say about the ...,The owner said that the accused did not have e...,The following provides an instruction in the f...
4,"Since 2022, how many agencies are responsible ...","Since 2022, six agencies will be responsible f...",The following provides an instruction in the f...
...,...,...,...
155,Are there other opportunities to become politi...,"Yes, there are other opportunities to become p...",The following provides an instruction in the f...
156,When and where does the City Night Bus CN1 go?,The City Night Bus CN1 goes from the Upper Tow...,The following provides an instruction in the f...
157,What was the main topic of the consultations o...,"The main topic was the refugee issue, especial...",The following provides an instruction in the f...
158,What is the role of the hereditary grand ducal...,We really live in very complicated times and y...,The following provides an instruction in the f...


In [ ]:
# Now we merge the dataframes back together
df = pd.concat([sampled_wiki_df, sampled_rtl_df])
df = df.reset_index(drop=True)
df

,instruction,response,text
0,What is the brightest star in Dorado?,"α Doradus, the brightest star in Dorado, is 20...",The following provides an instruction in the f...
1,"What happened to the Edict of Nantes, after He...","The Edict of Nantes was unfortunately not ""per...",The following provides an instruction in the f...
2,When did the devolution war start and how did ...,The war of devolution began in the spring of 1...,The following provides an instruction in the f...
3,When and where did Max Ophüls die?,"Max Ophüls died on March 26, 1957 in Hamburg.",The following provides an instruction in the f...
4,What happened to the Schumacher building after...,The police museum of the non-profit organizati...,The following provides an instruction in the f...
...,...,...,...
195,Are there other opportunities to become politi...,"Yes, there are other opportunities to become p...",The following provides an instruction in the f...
196,When and where does the City Night Bus CN1 go?,The City Night Bus CN1 goes from the Upper Tow...,The following provides an instruction in the f...
197,What was the main topic of the consultations o...,"The main topic was the refugee issue, especial...",The following provides an instruction in the f...
198,What is the role of the hereditary grand ducal...,We really live in very complicated times and y...,The following provides an instruction in the f...


In [ ]:
# check the prompt for correct format
for i in range(len(df[:5])):
  prompt = [{"role":"user","content":df["instruction"][i]},{"role":"assistant","content":df["response"][i]}]
  print(prompt)

[{'role': 'user', 'content': 'What is the brightest star in Dorado?'}, {'role': 'assistant', 'content': 'α Doradus, the brightest star in Dorado, is 200 light years away from us. The star is slightly white and belongs to the spectral class A0 III.'}]
[{'role': 'user', 'content': 'What happened to the Edict of Nantes, after Henri IV. died?'}, {'role': 'assistant', 'content': 'The Edict of Nantes was unfortunately not "perpétuel et irrevocable", which Louis XIV. evidenced by the Edict of Fontainebleau in 1685.'}]
[{'role': 'user', 'content': 'When did the devolution war start and how did it end?'}, {'role': 'assistant', 'content': 'The war of devolution began in the spring of 1667, when Louis XIV. sent a declaration to the Spanish court, and ended on May 2, 1668, when the Peace Treaty of Oachen was signed.'}]
[{'role': 'user', 'content': 'When and where did Max Ophüls die?'}, {'role': 'assistant', 'content': 'Max Ophüls died on March 26, 1957 in Hamburg.'}]
[{'role': 'user', 'content': '

# NIM PART - PROCEED WITH CAUTION !!!!!!!!!!!!!!!!!!!

In [ ]:
# !pip install -q openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00


In [ ]:
# from openai import OpenAI
# from google.colab import userdata

In [ ]:
# client = OpenAI(
#   base_url = "https://integrate.api.nvidia.com/v1",
#   api_key = userdata.get("NVIDIA_NIM")
# )

In [ ]:
# model_responses = []

In [ ]:
# for i in range(len(df)):

#   completion = client.chat.completions.create(
#     model="nvidia/nemotron-4-340b-reward",
#     messages=[{"role":"user","content":df["instruction"][i]},{"role":"assistant","content":df["response"][i]}],
#   )

#   # The nemotron model returns messages as a single element list containing
#   # ChatCompletionMessage, hence access with message[0].content instead of
#   # message.content like done in instruction models
#   answer = completion.choices[0].message[0].content
#   model_responses.append(answer)  # store model answer

In [ ]:
# model_responses

['helpfulness:1.84375,correctness:1.6640625,coherence:3.546875,complexity:1.0078125,verbosity:0.73046875',
 'helpfulness:2.75,correctness:2.890625,coherence:3.765625,complexity:1.1171875,verbosity:0.4765625',
 'helpfulness:2.578125,correctness:2.75,coherence:3.6875,complexity:1.1171875,verbosity:0.75390625',
 'helpfulness:3.171875,correctness:3.21875,coherence:3.984375,complexity:0.66796875,verbosity:0.484375',
 'helpfulness:2.890625,correctness:3.078125,coherence:3.75,complexity:1.140625,verbosity:1.1640625',
 'helpfulness:3.46875,correctness:3.5625,coherence:3.859375,complexity:1.078125,verbosity:0.76953125',
 'helpfulness:3.78125,correctness:3.796875,coherence:4.0,complexity:1.15625,verbosity:0.984375',
 'helpfulness:2.328125,correctness:2.515625,coherence:3.84375,complexity:1.15625,verbosity:0.81640625',
 'helpfulness:0.02294921875,correctness:-0.1982421875,coherence:2.78125,complexity:0.310546875,verbosity:0.27734375',
 'helpfulness:2.890625,correctness:2.859375,coherence:3.9375,c

# Save responses

In [ ]:
# model_responses

['helpfulness:1.84375,correctness:1.6640625,coherence:3.546875,complexity:1.0078125,verbosity:0.73046875',
 'helpfulness:2.75,correctness:2.890625,coherence:3.765625,complexity:1.1171875,verbosity:0.4765625',
 'helpfulness:2.578125,correctness:2.75,coherence:3.6875,complexity:1.1171875,verbosity:0.75390625',
 'helpfulness:3.171875,correctness:3.21875,coherence:3.984375,complexity:0.66796875,verbosity:0.484375',
 'helpfulness:2.890625,correctness:3.078125,coherence:3.75,complexity:1.140625,verbosity:1.1640625',
 'helpfulness:3.46875,correctness:3.5625,coherence:3.859375,complexity:1.078125,verbosity:0.76953125',
 'helpfulness:3.78125,correctness:3.796875,coherence:4.0,complexity:1.15625,verbosity:0.984375',
 'helpfulness:2.328125,correctness:2.515625,coherence:3.84375,complexity:1.15625,verbosity:0.81640625',
 'helpfulness:0.02294921875,correctness:-0.1982421875,coherence:2.78125,complexity:0.310546875,verbosity:0.27734375',
 'helpfulness:2.890625,correctness:2.859375,coherence:3.9375,c

In [ ]:
# df['reward'] = model_responses
# df

,instruction,response,text,model_response
0,What is the brightest star in Dorado?,"α Doradus, the brightest star in Dorado, is 20...",The following provides an instruction in the f...,"helpfulness:1.84375,correctness:1.6640625,cohe..."
1,"What happened to the Edict of Nantes, after He...","The Edict of Nantes was unfortunately not ""per...",The following provides an instruction in the f...,"helpfulness:2.75,correctness:2.890625,coherenc..."
2,When did the devolution war start and how did ...,The war of devolution began in the spring of 1...,The following provides an instruction in the f...,"helpfulness:2.578125,correctness:2.75,coherenc..."
3,When and where did Max Ophüls die?,"Max Ophüls died on March 26, 1957 in Hamburg.",The following provides an instruction in the f...,"helpfulness:3.171875,correctness:3.21875,coher..."
4,What happened to the Schumacher building after...,The police museum of the non-profit organizati...,The following provides an instruction in the f...,"helpfulness:2.890625,correctness:3.078125,cohe..."
...,...,...,...,...
195,Are there other opportunities to become politi...,"Yes, there are other opportunities to become p...",The following provides an instruction in the f...,"helpfulness:2.328125,correctness:2.53125,coher..."
196,When and where does the City Night Bus CN1 go?,The City Night Bus CN1 goes from the Upper Tow...,The following provides an instruction in the f...,"helpfulness:0.94140625,correctness:1.0390625,c..."
197,What was the main topic of the consultations o...,"The main topic was the refugee issue, especial...",The following provides an instruction in the f...,"helpfulness:2.671875,correctness:2.671875,cohe..."
198,What is the role of the hereditary grand ducal...,We really live in very complicated times and y...,The following provides an instruction in the f...,"helpfulness:1.171875,correctness:1.2734375,coh..."


In [ ]:
# df.to_csv('/content/drive/My Drive/Master Thesis/sampled_instruction_tuning_set_reward.csv', index=False)

# Import Data as Dataframe

In [ ]:
# # read in the csv file
# import pandas as pd
# df = pd.read_csv('/content/drive/My Drive/Master Thesis/sampled_instruction_tuning_set_reward.csv')
# df

,instruction,response,text,model_response
0,What is the brightest star in Dorado?,"α Doradus, the brightest star in Dorado, is 20...",The following provides an instruction in the f...,"helpfulness:1.84375,correctness:1.6640625,cohe..."
1,"What happened to the Edict of Nantes, after He...","The Edict of Nantes was unfortunately not ""per...",The following provides an instruction in the f...,"helpfulness:2.75,correctness:2.890625,coherenc..."
2,When did the devolution war start and how did ...,The war of devolution began in the spring of 1...,The following provides an instruction in the f...,"helpfulness:2.578125,correctness:2.75,coherenc..."
3,When and where did Max Ophüls die?,"Max Ophüls died on March 26, 1957 in Hamburg.",The following provides an instruction in the f...,"helpfulness:3.171875,correctness:3.21875,coher..."
4,What happened to the Schumacher building after...,The police museum of the non-profit organizati...,The following provides an instruction in the f...,"helpfulness:2.890625,correctness:3.078125,cohe..."
...,...,...,...,...
195,Are there other opportunities to become politi...,"Yes, there are other opportunities to become p...",The following provides an instruction in the f...,"helpfulness:2.328125,correctness:2.53125,coher..."
196,When and where does the City Night Bus CN1 go?,The City Night Bus CN1 goes from the Upper Tow...,The following provides an instruction in the f...,"helpfulness:0.94140625,correctness:1.0390625,c..."
197,What was the main topic of the consultations o...,"The main topic was the refugee issue, especial...",The following provides an instruction in the f...,"helpfulness:2.671875,correctness:2.671875,cohe..."
198,What is the role of the hereditary grand ducal...,We really live in very complicated times and y...,The following provides an instruction in the f...,"helpfulness:1.171875,correctness:1.2734375,coh..."
